In [1]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter


In [2]:
def filesave(dataframe,name):
    dataframe.to_csv(name,sep="\t", encoding="utf-8",index=False)
def fileread(name):
    return pd.read_csv(name,sep="\t",encoding="utf-8")

In [3]:
test=fileread('donga_dataframe.csv')

In [4]:
test

,title,content,date,section,url
0,"“NLL 침범 ‘무포호’, 과거 제3국서 등록 취소…최근까지 중국 입항” VOA",\n서해 북방한계선(NLL)을 침범했다 퇴각한 북한 상선 ‘무포호’가 과거 토고 등...,2022-10-25 06:58,Politics,https://www.donga.com/news/Politics/article/al...
1,"北, 中당대회 끝나자 서해NLL 도발… 7차 핵실험 ‘명분 쌓기’","\n먼저 월선하고도 “南이 도발” 트집\r\n北, 완충해역 포격… 9·19합의 위반...",2022-10-25 03:00,Politics,https://www.donga.com/news/Politics/article/al...
2,"檢, 민주硏 압수수색-정진상 출금… 민주 “野 말살”","\n檢, 김용 쓰던 PC서 문서파일 확보\r\n이틀째 金 불러 8억 용처 등 조사\...",2022-10-25 03:00,Politics,https://www.donga.com/news/Politics/article/al...
3,"이재명 “대장동 특검, 尹관련 저축銀 부분 빼도 좋다”",\n“김건희 특검과 무관” 수위 낮추며\r\n본인 수사 ‘檢 아닌 특검’ 뜻 내비쳐...,2022-10-25 03:00,Politics,https://www.donga.com/news/Politics/article/al...
4,"민주당, 오늘 尹대통령 국회 시정연설 보이콧… 尹 “시정연설에 조건, 헌정사서 들어...",\n박홍근 “당사 침탈 사과요구 무시”\r\n정진석 “시정연설은 국민 위한 것”\n...,2022-10-25 03:00,Politics,https://www.donga.com/news/Politics/article/al...
...,...,...,...,...,...
60297,펠레 위에 메시… 남미 A매치 79골 최다득점,\n리오넬 메시(왼쪽에서 두 번째)가 10일 아르헨티나 부에노스아이레스에서 열린 2...,2021-09-11 03:00,Sports,https://www.donga.com/news/Sports/article/all/...
60298,"‘현대가 더비’ 울산-전북, 득점없이 비겨",\n1위 울산-2위 전북 승점 차 4 유지\r\n대구는 포항 2-1로 꺾고 3위 올...,2021-09-11 03:00,Sports,https://www.donga.com/news/Sports/article/all/...
60299,"펠레 넘어선 메시, 남미 A매치 79골 최다득점",\n‘축구의 신’ 리오넬 메시(34·아르헨티나)가 ‘축구 황제’ 펠레(81·브라질)...,2021-09-10 21:10,Sports,https://www.donga.com/news/Sports/article/all/...
60300,“환골탈태가 필요하다”…국가대표 선수단 코치들 한목소리,\n‘한국 엘리트 스포츠의 환골탈태가 필요하다.’ 최근 열린 2020 도쿄 올림픽과...,2021-09-10 20:28,Sports,https://www.donga.com/news/Sports/article/all/...


In [5]:
test.drop_duplicates(inplace=True, ignore_index=True)

In [6]:
len(test)

59802

In [7]:
test['content'].nunique()

59315

In [8]:
def save_sentence_dataframe(data):
    idx=0
    sentences = []
    section_temp = []
    for i in range(len(data)):
        content = data['content'].iloc[idx]
        if "▲" in str(content):
            temp = str(content).split("▲")
            content=temp[0]
        splited = str(content).replace(". ","\n").split("\n")
        sentences += splited
        section_temp += [data['section'].iloc[idx]]*len(splited)
        idx+=1
    df = pd.DataFrame([sentences,section_temp]).T
    df.columns = ["sentence","section"]
    filesave(df, "donga_sentence_section.csv")

In [9]:
save_sentence_dataframe(test)

In [10]:
ttt=fileread("donga_sentence_section.csv")

In [210]:
def topic_counter_by_section(section,data):
    okt=Okt()
    parsed_data=[]
    sectiondata = data[data["section"]==section]
    for i in sectiondata['content']:
        if "▲" in str(i):
            temp = str(i).split("▲")
            i=temp[0]
        splited_list = str(i).replace(". ","\n").split("\n")
        if not splited_list:
            continue
        for sentence in splited_list:
            if len(sentence)<6:
                continue
            parsed_data += okt.pos(sentence, norm=True, stem=True)
    word_list = []
    
    stopword = ["없다","있다","재밌다","좋다","최고","같다","이","것","진짜","아니다","이렇다","그냥","하나","다시","많다","이다","최악",'완전',"안되다",'어떻다',"정말","보고",'정도','역시']
    for word,tag in parsed_data:
        if tag in ["Noun"] and len(word)>1 and word not in stopword:
            word_list.append(word)
    counts = Counter(word_list)
    return counts

In [21]:
def topic_counter_by_section_from_sentence_csv(section,data):
    okt=Okt()
    parsed_data=[]
    sectiondata = data[data["section"]==section]
    for sentence in sectiondata['sentence']:
        if len(str(sentence))<6:
            continue
        nouns = okt.nouns(str(sentence))
        stopword = ["없다","있다","재밌다","좋다","최고","같다","이","것","진짜","아니다","이렇다","그냥","하나","다시","많다","이다","최악",'완전',"안되다",'어떻다',"정말","보고",'정도','역시']
        for word in nouns:
            if len(word)>1 and word not in stopword:
                parsed_data.append(word)
    counts = Counter(parsed_data)
    return counts

In [212]:
def most_40topiclist_by_section(section,data):
    okt=Okt()
    parsed_data=[]
    sectiondata = data[data["section"]==section]
    for i in sectiondata["title"]:
        parsed_data+=okt.pos(i,norm=True,stem=True)

    word_list = []
    stopword = ["없다","있다","재밌다","좋다","최고","같다","이","것","진짜","아니다","이렇다","그냥","하나","다시","많다","이다","최악",'완전',"안되다",'어떻다',"정말","보고",'정도','역시']
    for word,tag in parsed_data:
        if tag in ["Noun"] and len(word)>1 and word not in stopword:
            word_list.append(word)


    counts = Counter(word_list)
    tags = counts.most_common(40)
    return tags

In [15]:
test["section"].value_counts()

Sports           9870
Society          9219
Inter            8947
Politics         8683
Culture          8247
Economy          8052
Entertainment    6784
Name: section, dtype: int64

In [12]:
sections = test["section"].value_counts()

In [13]:
sections = list(sections[sections>21].keys())

In [14]:
sections

['Sports',
 'Society',
 'Inter',
 'Politics',
 'Culture',
 'Economy',
 'Entertainment']

In [16]:
word_count_df = pd.DataFrame(columns=['word','count','section'])

In [17]:
word_count_df

,word,count,section


In [137]:
# counter=topic_counter_by_section("경제",test)

KeyboardInterrupt: 

In [ ]:
# temp = pd.DataFrame([dict(counter)]).T
# temp['word'] = temp.index
# temp['section'] = "경제"
# temp.columns=['count','word','section']
# word_count_df = pd.concat([word_count_df,temp],ignore_index=True)

In [ ]:
# word_count_df

In [18]:
def make_word_count_df():
    df = pd.DataFrame(columns=['word','count','section'])
    for section in sections:
        print(section)
        counter=topic_counter_by_section(section,test)
        temp = pd.DataFrame([dict(counter)]).T
        temp['word'] = temp.index
        temp['section'] = section
        temp.columns=['count','word','section']
        df = pd.concat([df,temp],ignore_index=True)
    return df

In [19]:
def make_word_count_df2(data,section_list):
    df = pd.DataFrame(columns=['word','count','section'])
    for section in section_list:
        print(section)
        counter=topic_counter_by_section_from_sentence_csv(section,data)
        temp = pd.DataFrame([dict(counter)]).T
        temp['word'] = temp.index
        temp['section'] = section
        temp.columns=['count','word','section']
        df = pd.concat([df,temp],ignore_index=True)
    return df

In [141]:
# most_40topiclist_by_section("경제",test)

In [30]:
t=test[(test['section']=="Sports")&(test['content'].str.contains("경기"))]

In [31]:
cnt=0
for i in t['content']:
    print(i)
    
    cnt+=1
    if cnt>6:
        break


K리그 2022 대상 시상식
울산 주장 맡아 우승의 숨은 주역… 선수들-코칭스태프 가교 역할 톡톡
막판 전북 추격에 흔들릴 때 다잡아
강원 양현준은 영플레이어상 수상
“요즘같이 살기 어려운 때 많은 분들이 포기하고 싶은 마음으로 살고 있다. 하지만 포기하지 않고 계속 도전하면 좋겠다. 우리 울산처럼.” 24일 서울 서초구 더케이호텔에서 열린 K리그 2022 대상 시상식에서 최우수선수(MVP) 상을 받은 이청용(34·울산)의 소감이다. K리그1(1부 리그) 울산은 2019시즌부터 지난 시즌까지 3년 연속을 포함해 준우승만 모두 10번 했는데 2005년 이후 17년 만인 올해 리그 정상에 다시 올랐다. 이청용은 K리그1 감독 12표 중 6표, 선수 12표 중 6표, 미디어투표 116표 중 59표를 얻었다. 감독(30%), 주장(30%), 미디어(40%) 투표 결과를 합산한 환산점수에서 50.34점을 기록했다. 이청용은 19.40점의 신진호(34·포항), 15.86점의 김대원(25·강원), 14.40점의 김진수(30·전북)를 제치고 K리그1 최고의 별이 됐다. 프로 데뷔 16년 만의 첫 MVP 수상이다.

  2006년 서울에서 프로 데뷔를 한 이청용은 2009년 당시 잉글랜드 프리미어리그 클럽이던 볼턴에 입단하며 유럽 무대에 진출했다. 이후 크리스털팰리스(잉글랜드), 보훔(독일)을 거쳤고 2020년 울산 유니폼을 입고 11년 만에 다시 K리그로 돌아왔다. 국가대표로는 2008년 5월 대표팀에 처음 발탁돼 2010년 남아프리카공화국, 2014년 브라질 월드컵에 출전했다. A매치 89경기에서 9골을 넣었다. 이청용은 지난해 홍명보 감독(53)이 울산 지휘봉을 잡은 뒤 팀 주장을 맡았다. 코칭스태프 회의에 참석하는 등 선수들과 코칭스태프 사이에서 가교 역할을 했다. 올 시즌 막판 2위 전북에 추격을 허용하며 팀 분위기가 어수선해졌을 때도 이청용이 나서 분위기를 다잡고 선수들을 독려했다. 울산 골키퍼 조현우(31)는 “청용이 형은 어떤 결정을 내려야 할 때 선수들과 공

In [22]:
w2 = make_word_count_df2(ttt,['Sports',
 'Society',
 'Inter',
 'Politics',
 'Culture',
 'Economy',
 'Entertainment'])

Sports
Society
Inter
Politics
Culture
Economy
Entertainment


In [23]:
w2

,word,count,section
0,리그,9151,Sports
1,대상,490,Sports
2,시상식,299,Sports
3,울산,1382,Sports
4,주장,709,Sports
...,...,...,...
226512,레저렉션,2,Entertainment
226513,백파,1,Entertainment
226514,뒤집힌다,1,Entertainment
226515,직수,1,Entertainment


In [24]:
w2.sort_values(by="count", ascending=False)

,word,count,section
80,경기,24483,Sports
93284,대통령,20882,Politics
93200,의원,19012,Politics
59678,러시아,18289,Inter
59439,미국,17618,Inter
...,...,...,...
117021,김빛,1,Politics
117020,병목,1,Politics
117018,강병구,1,Politics
19311,파찰음,1,Sports


In [25]:
w2['word_length']=[len(i) for i in w2['word']]

In [26]:
w2 = w2[w2['word_length']>1]

In [27]:
w2.drop(['word_length'],axis=1,inplace=True)

In [28]:
w2.sort_values(by="count", ascending=False)

,word,count,section
80,경기,24483,Sports
93284,대통령,20882,Politics
93200,의원,19012,Politics
59678,러시아,18289,Inter
59439,미국,17618,Inter
...,...,...,...
117021,김빛,1,Politics
117020,병목,1,Politics
117018,강병구,1,Politics
19311,파찰음,1,Sports


In [29]:
filesave(w2,"donga_content_word_count.csv")